# Analysis of Emergency Obstetric Care (EmOC) in Kano
> Note: All notebooks need the [environment dependencies](https://github.com/GIScience/openrouteservice-examples#local-installation)
> as well as an [openrouteservice API key](https://openrouteservice.org/dev/#/signup) to run

prepare environment dependencies document

## Abstract
The rapid growth of urban areas has put substantial pressure on local services and infrastructure, particularly in African cities. With migrants moving into cities and transient households moving within cities, traditional means of collecting data (e.g., censuses and household surveys) are inadequate and often overlook informal settlements and households. As a consequence, there is a chronic lack of basic data about deprived households and entire settlements. Given that urban poor residents rely predominantly on private and informal service providers for healthcare and other services, they are rarely captured in routine service data, including health information management systems. This is even more critical for women in need of maternal health care. 

Considering the different phases of maternity: antenatal care, interpartern or delivery, and postnatal care, the team decided to focus on interpartern or delivery phase being the most critical. The intertwined relationship between maternal health care and urban deprivation has been documented and described in the literature [Abascal et al., 2022](https://doi.org/10.1016/j.compenvurbsys.2022.101770). The IDEAMAPS Data Ecosystem team aims to analyse the conditions in which vulnerable communities relate to emergency maternal care (EmOC) in the city of Kano. To do so, the analysis is divided into three main components: 
1. **EmOC Offer**: Based on the geospatial database of travel times [(Macharia et al., 2023)](https://doi.org/10.1038/s41597-023-02651-9) and the team's field validation, we characterised 145 HC facilities offering EmOC in Kano, their service levels and relative costs.
2. **EmOC Accessibility**: The team used different routing services, including the OSM-based openrouteservice API, to calculate the travel times to the nearest EmOC facility for each 100x100m grid cell in Kano. 
3. **EmOC Demand**: The team discussed a set of socio-economic factors that determine the way communities from slums and other deprived areas demand or interact with EmOC services such as available income, employment, education, age, medical practitioners' age and gender as well as religious beliefs and social practices. despite not having access to specific data, the team discussed the potential impacts on demand for EmOC services in Kano based on these factors.



### Workflow:

The notebook gives an overview of the distribution of centres offering EmOC in Kano, their classification and how they can be accessed by car. Open source data from OpenStreetMap and tools (such as the openrouteservice) were used to create accessibility measures such as travel times and isochrones. Spatial analysis and other data analytics functions led to generating outputs within the 100x100m grid cells that categorised them into three levels: low, medium, and high.

* **Preprocessing**: Get data for EmOC facilities.
* **Analysis for Offer**:
    * Filter and classify EmOC facilities based on discussed criteria.
    * Visualise EmOC faccilities in their categories.
* **Analysis for Accessibility**:
    * Compute travel times to facilities using openrouteservice API or other routing services.
    * Generate areas for low, medium and high categories based on discussed criteria.
* **Analysis for Demmand**:
    * Derive socio-economic descriptors based on discussed criteria.
* **Result**: Visualize results as maps and export model outputs.


### Datasets and Tools:
* [A geospatial database of close to reality travel times to obstetric emergency care in 15 Nigerian conurbations](https://figshare.com/s/8868db0bf3fd18a9585d) - A curated list of health care facilities offering EmOC in Nigeria [(Macharia et al., 2023)](https://doi.org/10.1038/s41597-023-02651-9).
* [openrouteservice](https://openrouteservice.org/) - generate isochrones on the OpenStreetMap road network


# Python Workflow

This study integrates various Python geospatial analysis libraries and packages to support spatial data processing, visualization, and isochrone generation. The os module is used to interact with the operating system, managing file paths and reading environment variables such as API keys. folium library along with its MarkerCluster plugin, facilitates the creation of interactive maps for visualizing large-scale geospatial data. The openrouteservice.client serves as an interface to the OpenRouteService API, enabling the extraction of isochrones. pandas library for data analysis, provides functions for analyzing, cleaning, exploring, and manipulating data, while fiona supports reading and writing real-world data using multi-layered GIS formats, such as shapefiles. The shapely package is employed for the manipulation and analysis of planar geometric objects.

## Setting up the virtual environment

```bash
# Create a new virtual environment
python -m venv .venv
activate .venv/bin/activate
pip install -r requirements.txt
```

## To run your notebook in VS Code

```bash
pip install -U ipykernel
python -m ipykernel install --user --name=.venv
```

In [2]:
import os
from IPython.display import display
import requests

import folium
from folium.plugins import MarkerCluster
import openrouteservice
import time

import time
import pandas as pd
import numpy as np
import fiona as fn
import geopandas as gpd
from shapely.geometry import shape, mapping
from shapely.geometry import Point
from shapely.geometry import box
from scipy.spatial import cKDTree
from tqdm import tqdm

import rasterio
from rasterio.transform import xy
from rasterio.mask import mask
import rasterstats as rs

import math

## Preprocessing
In this study, users first requested an ORS Matrix API key from the [OpenRouteService](https://openrouteservice.org/) platform and subsequently interacted with the OpenRouteService API through the instantiation of the OpenRouteService client. This is the OpenRouteService [API documentation](https://openrouteservice.org/dev/#/api-docs/introduction) for ORS Core-Version 9.0.0. 

Generate a [API Key](https://openrouteservice.org/dev/#/home?tab=1) (Token) it is necessary to sign up at the OpenRouteService dashboard by using your E-mail address or sign up with your GitHub. After logging in, go to the Dashboard by clicking on your profile icon and navigate to the API Keys section. Click "Create API Key" to generate a free key and then choose a service plan (the free plan has limited requests per day). Copy the API Key and store it securely. 

OpenRouteService primarily uses API keys for authentication. However, if a token is required for certain endpoints, you can send a request with your API key in the Authorization header. This process facilitated various geospatial analysis functions, including isochrone generation.

### API Key
Make sure you have a .env file in the root directory with the following content:
```bash
    OPENROUTESERVICE_API_KEY='your_api_key'
```

In [3]:
# Read the api key from the .env file
from dotenv import load_dotenv
%load_ext dotenv
%dotenv
api_key = os.getenv('OPENROUTESERVICE_API_KEY')
client = openrouteservice.Client(key=api_key)

cannot find .env file


ValueError: No API key was specified. Please visit https://openrouteservice.org/sign-up to create one.

For this study different kind of data were used. The dataset on healthcare facilities is sourced from a research ([Macharia, P.M. et al., 2023](https://doi.org/10.1038/s41597-023-02651-9)) which provides A geospatial database of close-to-reality travel times to obstetric emergency care in 15 Nigerian conurbations. The dataset were filtered by state name to isolate facilities in Kano and converted CSV file to shapefile based on coordinates using [QGIS](https://qgis.org/). 

The Level 2 administrative boundary data is sourced from [Humanitarian Data Exchange](https://data.humdata.org/) were used to correlate the isochrones and healthcare facility distribution with specific administrative regions. The data were filtered based on the administrative region name (lganame) to focus the analysis on Kano.

Despite being official, administrative boundaries may not reflect the actual patterns of human settlement or economic activity. Therefore, the team used the Functional Urban Area (FUA) as a complementary definition of the study areas. The FUA is defined by [the Joint Research Centre of the European Commission](https://commission.europa.eu/about/departments-and-executive-agencies/joint-research-centre_en) as the actual urban sprawl and human activities, encompassing the core city and economically or socially integrated surrounding regions. The FUA was obtained from [the Global Human Settlement Layer (GHSL) ](https://human-settlement.emergency.copernicus.eu/)dataset, which provides spatial data for functional urban areas worldwide. 

* [Datasets of health facilities](https://doi.org/10.6084/m9.figshare.22689667.v2) (15/07/2023)
* [Shapefile of district boundaries](https://data.humdata.org/dataset/nigeria-admin-level-2) - Admin Level 2 (data from Humanitarian Data Exchange, 25/11/2015)
* [Functional Urban Areas](https://human-settlement.emergency.copernicus.eu/download.php?ds=FUA) - data from Global Human Settlement Layer(2015)

In [4]:
# Set paths to access data
# Define directories
data_inputs = '../scripts/data_inputs/'
data_temp = '../scripts/data_temp/'
data_outputs = '../scripts/data_outputs/'

## 1. Data Collection

### Validated healthcare facilities

In [5]:
healthcare_facilities = gpd.read_file(data_inputs + 'temp_hcf.gpkg')

### Population Grid Data (1km resolution) from WorldPop

In [6]:
raster_path = os.path.join(data_inputs, 'temp_pop.tif')

with rasterio.open(raster_path) as dataset:
    population_data = dataset.read(1)
    transform = dataset.transform

    rows, cols = np.where(population_data > 0)
    grid_cells = [[*transform * (col + 0.5, row + 0.5)] for row, col in zip(rows, cols)]
    population_values = population_data[rows, cols]

In [7]:
# Save the grid cells (centroids) with population data to a CSV
grid_df = pd.DataFrame(grid_cells, columns=["longitude", "latitude"])
grid_df['population'] = population_values  # Add the population count for each grid cell

# Generate unique grid codes
grid_df["grid_code"] = np.random.choice(range(10000, 99999), size=len(grid_df), replace=False)

# Save the DataFrame to GeoPackage
geometry = [Point(xy) for xy in zip(grid_df['longitude'], grid_df['latitude'])]
centroids_df = gpd.GeoDataFrame(grid_df, geometry=geometry)

centroids_df.set_crs('EPSG:4326', inplace=True)
gpkg_path = os.path.join(data_temp, 'population_centroids_temp.gpkg')
centroids_df.to_file(gpkg_path, driver="GPKG")

# Save the DataFrame to CSV
grid_csv_path = data_temp + 'population_centroids_temp.csv'
centroids_df.to_csv(grid_csv_path, index=False)

In [8]:
centroids_df

,longitude,latitude,population,grid_code,geometry
0,8.472917,12.024583,3054.140869,53937,POINT (8.47292 12.02458)
1,8.481250,12.024583,3737.392578,17840,POINT (8.48125 12.02458)
2,8.489583,12.024583,5439.910156,54264,POINT (8.48958 12.02458)
3,8.497917,12.024583,4532.464355,18673,POINT (8.49792 12.02458)
4,8.506250,12.024583,1577.261353,76531,POINT (8.50625 12.02458)
...,...,...,...,...,...
138,8.539583,11.941250,1152.257324,76416,POINT (8.53958 11.94125)
139,8.547917,11.941250,1221.242432,63699,POINT (8.54792 11.94125)
140,8.556250,11.941250,1514.486938,35666,POINT (8.55625 11.94125)
141,8.564583,11.941250,1321.376831,62382,POINT (8.56458 11.94125)


## 2. Spatial Analysis Pipeline
### Using OpenRouteService (ORS) Matrix API to calculate the travel time and distance from each population grid centroid to the healthcare facility 

In [9]:
origin_gdf = centroids_df
origin_name_column = 'grid_code'
destination_gdf = healthcare_facilities.dropna(subset=['geometry'])
destination_name_column = 'facility_name'

In [10]:
origins = list(zip(origin_gdf.geometry.x, origin_gdf.geometry.y))

In [11]:
destinations = list(zip(destination_gdf.geometry.x, destination_gdf.geometry.y))

In [12]:
locations = origins + destinations

In [13]:
origins_index = list(range(0, len(origins)))
destinations_index = list(range(len(origins), len(locations)))

In [18]:
body = {'locations': locations,
       'destinations': destinations_index,
       'sources': origins_index,
       'metrics': ['distance', 'duration']}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': api_key,
    'Content-Type': 'application/json; charset=utf-8'
}

response = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

In [19]:
distances = response.json().get('distances', [])
durations = response.json().get('durations', [])

In [20]:
distances_duration_matrix = []

# Iterate over each origin (grid)
for origin_index, origin in origin_gdf.iterrows():
    origin_name = origin[origin_name_column]
    origin_x = origin.geometry.x
    origin_y = origin.geometry.y
    origin_distances = distances[origin_index]
    origin_durations = durations[origin_index]

    # find the minimum duration and the index of the minimum duration
    min_duration = min(origin_durations)
    min_index = origin_durations.index(min_duration)
    destination_index = destinations_index[min_index]
    dest_x, dest_y = locations[destination_index]
    filtered = healthcare_facilities[(destination_gdf.geometry.x == dest_x) & (destination_gdf.geometry.y == dest_y) ]
    destination_row = filtered.iloc[0]
    dest_name = destination_row[destination_name_column]

        # Append both the distance and duration for this origin-destination pair
    distances_duration_matrix.append([
            origin_name, origin_y, origin_x,
            dest_name, dest_y, dest_x,
            min_duration
        ])

# Convert the results into a DataFrame
matrix_df = pd.DataFrame(distances_duration_matrix, columns=[
    'grid_code','origin_lat', 'origin_lon',
    'destination_name', 'dest_lat', 'dest_lon','min_duration'
])

In [23]:
# Save to CSV
merged_df = pd.merge(matrix_df, grid_df[['grid_code', 'population']], on='grid_code', how='left')
merged_df.to_csv(data_temp + 'distance_duration_matrix_temp.csv', index=False)

In [24]:
merged_df

,grid_code,origin_lat,origin_lon,destination_name,dest_lat,dest_lon,min_duration,population
0,53937,12.024583,8.472917,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,686.79,3054.140869
1,17840,12.024583,8.481250,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,722.17,3737.392578
2,54264,12.024583,8.489583,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,569.70,5439.910156
3,18673,12.024583,8.497917,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,395.79,4532.464355
4,76531,12.024583,8.506250,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,411.43,1577.261353
...,...,...,...,...,...,...,...,...
138,76416,11.941250,8.539583,Maxcare Clinic,11.96792,8.54314,543.74,1152.257324
139,63699,11.941250,8.547917,Maxcare Clinic,11.96792,8.54314,419.18,1221.242432
140,35666,11.941250,8.556250,Maxcare Clinic,11.96792,8.54314,295.33,1514.486938
141,62382,11.941250,8.564583,Maxcare Clinic,11.96792,8.54314,265.40,1321.376831


In [25]:
geometry = [Point(xy) for xy in zip(merged_df['dest_lon'], merged_df['dest_lat'])]
gdf = gpd.GeoDataFrame(merged_df, geometry=geometry, crs="EPSG:4326")

gpkg_path = data_temp + 'distance_duration_matrix_temp.gpkg'
gdf.to_file(gpkg_path, layer="duration_matrix", driver="GPKG")

### Option: to not request OD Matrix. Read the Existing CSV File

In [26]:
matrix_df = pd.read_csv(data_temp +'distance_duration_matrix_temp.csv')

In [27]:
# Review and remove
origin_dest = matrix_df

## Enhanced Two-Step Floating Catchment Area (E2SFCA) method

In [28]:
# Function
from math import *
d = 10 * 60 # try max duration 10mins car
W = 0.1
beta = - d ** 2 / log(W)
print(beta)

156346.01348517067


In [29]:
# Generate a unique code for Each HCF
origin_dest['unique_code'] = origin_dest[['dest_lon', 'dest_lat']].apply(lambda x: hash(tuple(x)), axis=1)

In [30]:
print(origin_dest.head())

   grid_code  origin_lat  origin_lon                  destination_name  \
0      53937   12.024583    8.472917  Sabo Bakin Zuwo General Hospital   
1      17840   12.024583    8.481250  Sabo Bakin Zuwo General Hospital   
2      54264   12.024583    8.489583  Sabo Bakin Zuwo General Hospital   
3      18673   12.024583    8.497917  Sabo Bakin Zuwo General Hospital   
4      76531   12.024583    8.506250  Sabo Bakin Zuwo General Hospital   

   dest_lat  dest_lon  min_duration  population          unique_code  
0  12.00065   8.50923        686.79   3054.1409  5030447890719996479  
1  12.00065   8.50923        722.17   3737.3926  5030447890719996479  
2  12.00065   8.50923        569.70   5439.9100  5030447890719996479  
3  12.00065   8.50923        395.79   4532.4644  5030447890719996479  
4  12.00065   8.50923        411.43   1577.2614  5030447890719996479  


In [31]:
# Convert 'duration' to numeric, coercing errors to NaN
origin_dest['min_duration'] = pd.to_numeric(origin_dest['min_duration'], errors='coerce')

# Drop rows with NaN values in 'duration' column
origin_dest = origin_dest.dropna(subset=['min_duration'])
origin_dest['grid_code'] = pd.to_numeric(origin_dest['grid_code'], errors='coerce')

origin_dest_acc = origin_dest  # Backup

In [32]:
# Apply Gaussian decay function to calculate the weight of each grid to healthcare facilities based on the travel duration. d is the travel time and beta is the decay parameter previously calculated.
# The weight decreases as the duration increases, meaning facilities that are further away have less impact.
origin_dest_acc['Weight'] = origin_dest_acc['min_duration'].apply(lambda d: round(math.exp(-d**2/beta), 8))

In [33]:
# Compute the Weighted Population (Pop_W), the population of each grid cell is multiplied by the corresponding weight to calculate the weighted population.
origin_dest_acc['Pop_W'] = origin_dest_acc['population'] * origin_dest_acc['Weight']

In [34]:
origin_dest_acc

,grid_code,origin_lat,origin_lon,destination_name,dest_lat,dest_lon,min_duration,population,unique_code,Weight,Pop_W
0,53937,12.024583,8.472917,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,686.79,3054.1409,5030447890719996479,0.048953,149.508352
1,17840,12.024583,8.481250,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,722.17,3737.3926,5030447890719996479,0.035588,133.007337
2,54264,12.024583,8.489583,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,569.70,5439.9100,5030447890719996479,0.125444,682.403852
3,18673,12.024583,8.497917,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,395.79,4532.4644,5030447890719996479,0.367166,1664.164603
4,76531,12.024583,8.506250,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,411.43,1577.2614,5030447890719996479,0.338682,534.190708
...,...,...,...,...,...,...,...,...,...,...,...
138,76416,11.941250,8.539583,Maxcare Clinic,11.96792,8.54314,543.74,1152.2573,-4123203778311363183,0.150918,173.896413
139,63699,11.941250,8.547917,Maxcare Clinic,11.96792,8.54314,419.18,1221.2424,-4123203778311363183,0.325021,396.929463
140,35666,11.941250,8.556250,Maxcare Clinic,11.96792,8.54314,295.33,1514.4869,-4123203778311363183,0.572430,866.938463
141,62382,11.941250,8.564583,Maxcare Clinic,11.96792,8.54314,265.40,1321.3768,-4123203778311363183,0.637296,842.108202


In [35]:
# Sum the Weighted Population
origin_dest_sum = origin_dest_acc.groupby(by='unique_code')['Pop_W'].sum().reset_index()

In [36]:
origin_dest_sum 

,unique_code,Pop_W
0,-4123203778311363183,45095.588621
1,3419389286098593137,1173.918201
2,5030447890719996479,73133.214392
3,6469257637603030174,110237.045483


In [37]:
# Merge the Sum of Weighted Population Back into the Original Data
origin_dest_acc = origin_dest_acc.merge(origin_dest_sum, on='unique_code')

In [38]:
origin_dest_acc

,grid_code,origin_lat,origin_lon,destination_name,dest_lat,dest_lon,min_duration,population,unique_code,Weight,Pop_W_x,Pop_W_y
0,53937,12.024583,8.472917,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,686.79,3054.1409,5030447890719996479,0.048953,149.508352,73133.214392
1,17840,12.024583,8.481250,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,722.17,3737.3926,5030447890719996479,0.035588,133.007337,73133.214392
2,54264,12.024583,8.489583,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,569.70,5439.9100,5030447890719996479,0.125444,682.403852,73133.214392
3,18673,12.024583,8.497917,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,395.79,4532.4644,5030447890719996479,0.367166,1664.164603,73133.214392
4,76531,12.024583,8.506250,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,411.43,1577.2614,5030447890719996479,0.338682,534.190708,73133.214392
...,...,...,...,...,...,...,...,...,...,...,...,...
138,63699,11.941250,8.547917,Maxcare Clinic,11.96792,8.54314,419.18,1221.2424,-4123203778311363183,0.325021,396.929463,45095.588621
139,35666,11.941250,8.556250,Maxcare Clinic,11.96792,8.54314,295.33,1514.4869,-4123203778311363183,0.572430,866.938463,45095.588621
140,62382,11.941250,8.564583,Maxcare Clinic,11.96792,8.54314,265.40,1321.3768,-4123203778311363183,0.637296,842.108202,45095.588621
141,93998,11.941250,8.572917,Maxcare Clinic,11.96792,8.54314,403.17,993.2351,-4123203778311363183,0.353576,351.184431,45095.588621


In [39]:
# supply value is set to 1 for simplicity (capacity of HCF)
supply = 1
origin_dest_acc = origin_dest_acc.rename(columns={'Pop_W_y': 'Pop_W_S'})  # Pop_W_S: Population Weight Sum

In [40]:
# Compute the Supply-Demand Ratio (Rj)
origin_dest_acc['supply_demand_ratio'] = 1 / origin_dest_acc.Pop_W_S
origin_dest_acc['supply_demand_ratio'].replace([np.inf, np.nan], 0, inplace=True)

In [41]:
# Calculate Rj * Weight for Each Grid Cell
origin_dest_acc['supply_W'] = origin_dest_acc['supply_demand_ratio'] * origin_dest_acc.Weight

In [42]:
# Compute Accessibility Index (Ai) for Each Grid Cell
origin_dest_acc['Accessibility'] = origin_dest_acc.groupby('grid_code')['supply_W'].transform('sum')

In [43]:
# Normalize
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
origin_dest_acc['Accessibility_standard'] = scaler.fit_transform(origin_dest_acc[['Accessibility']])

In [44]:
origin_dest_acc

,grid_code,origin_lat,origin_lon,destination_name,dest_lat,dest_lon,min_duration,population,unique_code,Weight,Pop_W_x,Pop_W_S,supply_demand_ratio,supply_W,Accessibility,Accessibility_standard
0,53937,12.024583,8.472917,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,686.79,3054.1409,5030447890719996479,0.048953,149.508352,73133.214392,0.000014,6.693630e-07,6.693630e-07,0.000788
1,17840,12.024583,8.481250,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,722.17,3737.3926,5030447890719996479,0.035588,133.007337,73133.214392,0.000014,4.866225e-07,4.866225e-07,0.000452
2,54264,12.024583,8.489583,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,569.70,5439.9100,5030447890719996479,0.125444,682.403852,73133.214392,0.000014,1.715280e-06,1.715280e-06,0.002713
3,18673,12.024583,8.497917,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,395.79,4532.4644,5030447890719996479,0.367166,1664.164603,73133.214392,0.000014,5.020503e-06,5.020503e-06,0.008796
4,76531,12.024583,8.506250,Sabo Bakin Zuwo General Hospital,12.00065,8.50923,411.43,1577.2614,5030447890719996479,0.338682,534.190708,73133.214392,0.000014,4.631034e-06,4.631034e-06,0.008080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,63699,11.941250,8.547917,Maxcare Clinic,11.96792,8.54314,419.18,1221.2424,-4123203778311363183,0.325021,396.929463,45095.588621,0.000022,7.207380e-06,7.207380e-06,0.012822
139,35666,11.941250,8.556250,Maxcare Clinic,11.96792,8.54314,295.33,1514.4869,-4123203778311363183,0.572430,866.938463,45095.588621,0.000022,1.269371e-05,1.269371e-05,0.022919
140,62382,11.941250,8.564583,Maxcare Clinic,11.96792,8.54314,265.40,1321.3768,-4123203778311363183,0.637296,842.108202,45095.588621,0.000022,1.413211e-05,1.413211e-05,0.025567
141,93998,11.941250,8.572917,Maxcare Clinic,11.96792,8.54314,403.17,993.2351,-4123203778311363183,0.353576,351.184431,45095.588621,0.000022,7.840597e-06,7.840597e-06,0.013987


In [45]:
max(origin_dest_acc.Accessibility_standard)

0.9999999999999999

In [46]:
geometry = [Point(xy) for xy in zip(origin_dest_acc['origin_lon'], origin_dest_acc['origin_lat'])]
gdf = gpd.GeoDataFrame(origin_dest_acc, geometry=geometry, crs="EPSG:4326")

gpkg_path = data_outputs + 'origin_dest_acc.gpkg'
gdf.to_file(gpkg_path, layer="origin_dest_acc", driver="GPKG")